In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from urllib import parse
import time
import pandas as pd
import re
from tqdm import tqdm

In [8]:
# TAAS GIS 시스템 로드
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://taas.koroad.or.kr/gis/mcm/mcl/initMap.do?menuId=GIS_GMP_STS_RSN#')
time.sleep(1)




In [9]:
# 체크박스 체크
selectbox0 = driver.find_element(By.ID, 'ptsRafYearStart')
select = Select(selectbox0)
select.select_by_value("2017") # 2017년부터

elements = driver.find_elements(By.NAME, 'ACDNT_GAE_CODE')[0:4]
for element in elements:
    if not element.is_selected():
        element.click()

selectbox1 = driver.find_element(By.ID, 'ptsRafSigungu')
select = Select(selectbox1)
select.select_by_value("01") # 전체 지역구(서울)

selectbox2 = driver.find_element(By.ID, 'ptsRafSimpleCondition')
select = Select(selectbox2)
select.select_by_value("44") # 보행 어린이 사고

# vehicle = driver.find_element(By.XPATH, '//*[@id="ptsRaf-WRNGDO_VHCLE_ASORT_CODE"]/a') # 가해차종 > PM만 체크
# vehicle.click()
time.sleep(1)



In [10]:
# search
driver.find_element(By.XPATH, '//*[@id="regionAccidentFind"]/div[2]/p/a').send_keys(Keys.ENTER)
time.sleep(1)


In [12]:
# Crawling
coord_ls = []
srs_ls = []

for i in tqdm(range(0,1805)):
    driver.execute_script(f'gis.srh.msh.selectListUi({i}, 11);') # 개별 사고지점
    time.sleep(1) # 로드
    bbox = driver.find_elements(By.CLASS_NAME,'olTileImage')[-1]
    src = parse.unquote(bbox.get_attribute('src')) # get src and unquote from 16byte
    src_split = src.split('&')
    bbox_text = src_split[-3][5:]
    coords = [float(i) for i in bbox_text.split(',')]
    x, y = (coords[0]+coords[2])/2, (coords[1]+coords[3])/2 # center position
    srs = src_split[-4][4:]
    coord_ls.append([x, y])
    srs_ls.append(srs)

100%|██████████| 1805/1805 [32:10<00:00,  1.07s/it]


In [13]:
# 크롤링한 좌표 확인
coord_ls

[[206991.27600645, 454598.54931228],
 [193632.16820104, 443290.1259301],
 [204196.20022334, 456471.41481355],
 [205449.32310377, 446199.01736629],
 [210683.3722807, 448652.80771713],
 [207802.30751371, 453901.58730183],
 [203500.70461001, 458766.61667473],
 [205861.396306, 453175.97266046],
 [206140.68764757, 444110.86164473],
 [206766.88062107, 456347.05029276],
 [205200.35883922, 444290.93973966],
 [208804.71923665, 456529.97144653],
 [211844.47989879, 447779.62701899],
 [185919.60433466, 443599.29335361],
 [204624.55736477, 453617.56733108],
 [197229.61929472, 453438.1537602],
 [202082.21809638, 443521.07965472],
 [199090.6623146, 444535.59088846],
 [194255.18108606, 444650.95593995],
 [194255.00826389, 441669.8109213],
 [206475.33166459, 462435.91711731],
 [201613.96599163, 457827.19971219],
 [191848.18310196, 447677.32661249],
 [192903.15580922, 457533.70213121],
 [186844.67189697, 448357.99326574],
 [185086.40237342, 443766.93195973],
 [193200.73013917, 454817.24679962],
 [201213

In [14]:
# 크롤링데이터 json 파일로 저장
import json

# JSON 파일로 저장
with open('child_accident_coord_data.json', 'w') as json_file:
    json.dump(coord_ls, json_file)

In [16]:
# 크롤링한 좌표 데이터 갯수 체크
import json

# 파일 이름
file_name = 'child_accident_coord_data.json'

# 파일 내용 읽기
with open(file_name, 'r') as file:
    json_data = json.load(file)

# JSON 데이터의 개수 확인
data_count = len(json_data)

# 결과 출력
print(f"{file_name} 파일의 데이터 개수: {data_count}")

child_accident_coord_data.json 파일의 데이터 개수: 1805


In [19]:
# 크롤링데이터 csv파일로 변환
import pandas as pd

# JSON 파일 읽기
with open('child_accident_coord_data.json', 'r') as json_file:
    data = json.load(json_file)

# JSON 데이터를 DataFrame으로 변환
df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv('서울시어린이보행자교통사고좌표값.csv', index=False)